In [45]:
from konlpy.tag import Twitter
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import normalize
from scipy.sparse import dok_matrix
from random import shuffle
import numpy as np
import json

twitter = Twitter()

In [46]:
with open("data/2017_03.json") as f:
    data = json.load(f)

In [34]:
data[0]['date']

'2017-03-01'

In [47]:
shuffle(data)
data = data[:5000]

In [48]:
titles = set()
articleList = []
articleNum = 0

for article in data:
    if article['title'] not in titles:
        # 기사 중복되지 않는 것들만 처리
        titles.add(article['title'])
        articleList.append(article['text'])
        articleNum += 1

del data

In [49]:
articleList[0]

"▲더불어민주당 대선주자인 최성 고양시장(왼쪽부터), 안희정 충남지사, 사회자 정관용, 이재명 성남시장, 문재인 전 대표가 3일 오후 서울 목동 CBS사옥에서 시사프로그램인 '시사자키 정관용입니다'에 출연, 합동토론회 준비를 하고 있다.(사진=연합뉴스)\n\n 더불어민주당 대선주자 첫 합동토론회에서는 박근혜 대통령에 대한 사법처리 문제가 화두로 올랐다.\n\n문재인 전 대표, 안희정 충남도지사, 이재명 성남지사, 최성 고양시장은 3일 오후 CBS 주최로 열린 합동토론회에서 헌법재판소에서 박근혜 대통령에 대한 탄핵이 인용될 경우 박 대통령에 대한 수사와 사법처리를 묻는 질문에 “엄정한 법적 심판”에 한목소리를 냈다.\n\n가장 먼저 답변 기회를 얻은 최성 시장은 “대한민국의 불행한 역사가 오게 된 계기 중에 하나는 일제 식민시대의 유산을 청산하지 못하고 박정희 군사정권 시절의 유산을 청산하지 못한 측면이라고 본다”고 지적했다.\n\n최 시장은 “이번에 박근혜 대통령에 대한 탄핵이 이루어지더라도 이 과정에서 벌어졌던 국정농단의 실태들, 정말 모든 부분에서 법적으로 판단해서 역사에 이런 비극적 사태가 재현되지 않는 엄정한 법적 심판을 내려야 한다”고 밝혔다.\n\n안희정 지사도 “헌법과 법률의 정신대로 진행된 수사는 수사대로 진행해야 한다. 우리는 법 위에 어떠한 특권 세력도 존재할 수 없다”며 “그런 점에서 일체의 정치적 불법사실에 대해서는 정치적 타협과 해법의 논의를 거부한다”고 밝혔다.\n\n안 지사는 “헌법과 법률의 정신으로 가자. 제가 늘 삼권분립을 얘기하고 법치 정신을 강조하는 이유”라며 “이제 더 이상 모든 어떤 사건들을 정치적 봉합이라는 이름으로는 처리하지 않을 계획이다”고 말했다.\n\n안 지사 다음으로 답변에 나선 이재명 시장은 “당연히 보통 사람과 똑같이 대통령 퇴임과 동시에 구속하고 엄정하게 처벌해야 된다”고 강한 어조로 말했다.\n\n이 시장은 “대한민국은 민주공화국이고 민주공화국의 관점에서 가장 중요한 건 법 앞에 만인이 평등하다는 것”이라

In [37]:
tags = set(['Noun', 'Verb', 'Adjective'])
spamWords = set([
    '하다', '되다', '이다', '전재', '배포', '무단', '기사', '금지', '사진', '보기', '뉴스', '있다',
    '돼다', '앵커', '위해', '라며', '그렇다', '보다', '되어다', '기자', '하는', '이런', '그런', '하는',
    '하였', '있었', '입니'
])
wordsList = []
index2voca = set()
for i, article in enumerate(articleList):
    if i % 1000 == 0:
        print(i)

    words = []
    p = twitter.pos(article, norm=True)
    for word, tag in p:
        if tag in tags and len(word) > 1 and word not in spamWords:
            words.append(word)
            index2voca.add(word)

    wordsList.append(words)

index2voca = list(index2voca)
voca2index = {w: i for i, w in enumerate(index2voca)}

print(len(index2voca))
print(len(wordsList))

0
1000
2000
3000
4000
36468
4932


In [54]:
voca2index['서울']

10232

In [38]:
# term-document matrix를 sparse matrix로 생성
tdm = dok_matrix((len(wordsList), len(index2voca)), dtype=np.float32)
print(tdm.shape)

for i, words in enumerate(wordsList):
    for word in words:
        tdm[i, voca2index[word]] += 1

tdm = tdm.tocsr()
# 각 document별로 l2-normalize
tdm_ = normalize(tdm)

(4932, 36468)


In [55]:
# LDA
K = 10
lda = LatentDirichletAllocation(n_components=K, learning_method='batch')
W = lda.fit_transform(tdm_)
H = lda.components_

# 각 토픽별 키워드 출력
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(index2voca[index], end=" ")
    print("\n")

0th topic
입력 수정 전체 접속 발행 등록 되었 다한 차단 이상면 번호 청소년 편집 보호책임 빌딩 서울시 서울 제호 계동 일자 

1th topic
정기 시스템 뷰스앤뉴스 매일 점검 포천시 깨지 독점 보수당 배구 꽃놀이패 김동성 표지 명리 이택광 홍성 바흐 허씨 삼청교육대 신광 

2th topic
탄핵 대통령 박근혜 국민 대표 서울 집회 대선 의원 한국 결정 민주당 있는 인용 대한 태극기 헌재 우리 오후 말했 

3th topic
켈리 방송사고 웃음 스타 사탕 부부 행복 등장 그래프 실시간 뜨는 인기 사람 시각 접근하는 많은 지금 나타낸 얼마나 텔레 

4th topic
시간 대선 채널 지원 억원 출마 보통사람 정부 청춘 황교안 소환 검찰 인상 사이트 금리 문재인 공개 변경 주택 웨스턴 

5th topic
접속 귀하 읽으 여기 계속 되었 주세 다한 클릭 하신 차단 최신 완전체 힘찬 연예 방용국 어려운 유명 논란 고민 

6th topic
대통령 탄핵 박근혜 선고 심판 헌재 수사 헌법재판소 서울 청와대 결정 특검 검찰 재판관 대한 대행 경찰 권한 인용 파면 

7th topic
대구 도시 경북 성소수자 스토리 낙동강 물의 찾아 유머 명소 기행 강국 라이온즈 가다 권현구 피셔 수익률 반포 개막전 관중 

8th topic
많아 당한 사기 고백 얼마 박시연 주군 공양 국운 전셋값 아듀 병신년 구주 이모저모 성탄절 정유년 오셨 귀성 명절 소리치 

9th topic
공준 김성욱 찔림 소소한 절박 백승빈 울릉도 수화 피셔 나음 상함 채찍 로보 장수영 김종배 오상진 통역사 도시바 출가 뉴저지 



In [40]:
from sklearn.manifold import TSNE
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [56]:
# select random index
selectNum = 1000
randIndex = np.random.choice(W.shape[0], selectNum, replace=False)
randIndex.sort()

tsne = TSNE(n_components=2, init='pca', verbose=1)
W2d = tsne.fit_transform(W[randIndex, :])
topicIndex = [v.argmax() for v in W[randIndex, :]]


# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=800, plot_height=500, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in randIndex],
    'document': [articleList[randInd][:50] for randInd in randIndex],  # 해당 기사
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color',
    fill_alpha=0.7,
    line_alpha=0.7)

# 몇가지 interaction
p.legend.location = "top_left"
hover = p.select(dict(type=HoverTool))
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.001s...
[t-SNE] Computed neighbors for 1000 samples in 0.028s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 47.398952
[t-SNE] Error after 1000 iterations: 0.057213


In [57]:
# NMF
K = 10
nmf = NMF(n_components=K, max_iter=1000, alpha=0.1)
W = nmf.fit_transform(tdm_)
H = nmf.components_

# 각 토픽별 키워드 출력
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(index2voca[index], end=" ")
    print("\n")

0th topic
대통령 박근혜 청와대 탄핵 파면 삼성동 사저 결정 의원 인용 검찰 헌법재판소 대한 입장 국민 지지자 없다 지난 있는 변호사 

1th topic
켈리 방송사고 웃음 스타 부부 행복 등장 사탕 교수 아이 인터뷰 로버트 부산 영국 방송 동영상 아내 아들 영상 모습 

2th topic
대표 의원 대선 민주당 후보 정당 한국 탄핵 지사 더불어 문재인 바른 자유 경선 국민 주자 탈당 국민의당 국회 말했 

3th topic
집회 탄핵 태극기 경찰 국민 반대 기각 참가자 운동 본부 궐기 위한 오후 단체 행진 대통령 주최 광장 촛불 시위 

4th topic
시간 대선 지원 출마 억원 보통사람 인상 검찰 소환 정부 청춘 황교안 변경 금리 공개 문재인 주택 한국 출시 이상 

5th topic
탄핵 선고 심판 헌재 헌법재판소 결정 재판관 대통령 인용 오전 박근혜 기각 대한 국회 평의 결과 사건 의견 기일 각하 

6th topic
사드 국민 중국 우리 경제 한국 정부 있는 배치 대한 문제 정치 탄핵 미국 상황 결정 대한민국 말했 북한 국가 

7th topic
수사 특검 검찰 결과 혐의 박영수 사실 발표 대통령 최순실 대한 조사 특별검사 사건 삼성 대해 의혹 변호사 내용 뇌물 

8th topic
서울 박근혜 오후 광장 탄핵 광화문 열린 촛불집회 퇴진 행동 인용 헌법재판소 종로구 시민 뉴시스 국민 촛불 참가자 정권 하고 

9th topic
대행 권한 대선 대통령 출마 황교안 재판관 정미 선거 퇴임 헌재 후보 국정 소장 상황 헌법재판소장 선거일 국무회의 지정 있는 



In [58]:
# select random index
selectNum = 1000
randIndex = np.random.choice(W.shape[0], selectNum, replace=False)
randIndex.sort()

tsne = TSNE(n_components=2, init='pca', verbose=1)
W2d = tsne.fit_transform(W[randIndex, :])
topicIndex = [v.argmax() for v in W[randIndex, :]]


# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=800, plot_height=500, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in randIndex],
    'document': [articleList[randInd][:50] for randInd in randIndex],  # 해당 기사
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color',
    fill_alpha=0.7,
    line_alpha=0.7)

# 몇가지 interaction
p.legend.location = "top_left"
hover = p.select(dict(type=HoverTool))
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.001s...
[t-SNE] Computed neighbors for 1000 samples in 0.056s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.730629
[t-SNE] Error after 1000 iterations: 0.666249


In [59]:
articleList[2275]

'  ŔĚľĽŔĎ¸Ž°Ą ĂźŔÎÁöÄÚ¸ŽžĆ ˝Ă¸ŽÁîŔÇ ÇĎłŞˇÎ ¸śˇĂÇŃ °łÇĺ ÁÂ´ăČ¸. ŔĚťóľˇ ąššÎŔÇ´ç ŔÇżř, ˝ĹŔ˛ ¸íÁö´ë ąłźö, ąčźşĹÂ ŔÚŔŻÇŃąš´ç ŔÇżřŔĚ ĹäˇĐŔť ÇĎ°í ŔÖ´Ů.\n\n  ŔĚťóľˇ ąššÎŔÇ ´ç ŔÇżřŔĚ ż\xadşŻŔť ĹäÇĎ°í ŔÖ´Ů.\n\n  °łÇĺ ÁÂ´ăČ¸. ąčźşĹÂ ŔÚŔŻÇŃąš´ç ŔÇżřŔĚ šßžđŔť ÇĎ°í ŔÖ´Ů.\n\n  °łÇĺ ÁÂ´ăČ¸. ˝ĹŔ˛ ¸íÁö´ë ąłźö°Ą °łÇĺŔÇ ÇĘżäźşŔť żŞźłÇĎ°í ŔÖ´Ů.\n\n\n\nXML:Y\n\n  šÚąŮÇý Ŕü ´ëĹëˇÉŔĚ ÇĺšýŔçĆÇźŇżĄ ŔÇÇŘ ĆÄ¸éľÇ¸éź\xad 1987łâ °łÇĺ ŔĚČÄ ľîŔĺÇŃ ´ëĹëˇÉ 6¸íŔĚ ¸đľÎ şŇÇŕÇŃ ¸ťłâŔť ¸ÂŔĚÇß´Ů. ŔĚ¸Ś żîŔüŔÚ(´ëĹëˇÉ) °ú˝ÇˇÎ şź °ÍŔÎ°Ą, ŔÚľżÂ÷(ąš°Ą ąÇˇÂ˝Ă˝şĹŰ) °áÇÔŔ¸ˇÎ şÁžß ÇŇ °ÍŔÎ°Ą. °łÇĺ šŽÁŚ´Â Á¤ÄĄąÇŔÇ ÇÖŔĚ˝´ˇÎ şÎťóÇßŔ¸¸ç, ąšČ¸ľľ ŔĚ°°Ŕş šŽÁŚŔÇ˝ÄŔť °Ž°í °łÇĺĆŻŔ§¸Ś °ĄľżÁßŔĚ´Ů.ŔĚľĽŔĎ¸Ž´Â Áöł\xad 9ŔĎ °łÇĺŔť ÁÖÁŚˇÎ ÁÂ´ăČ¸¸Ś °ĄÁł´Ů. şťÁö Á¤°ćşÎŔĺŔÎ ąčČ\xadąŐ şÎąšŔĺŔÇ ťçČ¸ˇÎ, °łÇĺĆŻŔ§ 1źŇŔ§żÍ 2źŇŔ§ °ŁťçŔÎ ąčźşĹÂ ŔÚŔŻÇŃąš´ç ŔÇżř°ú ŔĚťóľˇ ąššÎŔÇ´ç ŔÇżř, Á¤ÄĄĆňˇĐ°ĄŔÎ ˝ĹŔ˛ ¸íÁö´ë ąłźö°Ą °˘ŔÚŔÇ °ßÇŘ¸Ś šŕÇű´Ů. Âü°ĄŔÚľéŔş °łÇĺŔÇ ´çŔ§źşżĄ °ř°¨ÇĎ¸éź\xadľľ °łÇĺŔÇ šß¸ńŔť Ŕâ°í ŔÖ´Â šŽÁŚÁĄŔť Áś¸ńÁś¸ń Â¤žú´Ů. ´ëĹëˇÉ ĹşÇŮ ˝Ă´ë¸Ś ¸ÂŔĚÇŘ żě¸Ž ťçČ¸°